# Ray Aiming Tutorial

This tutorial demonstrates the new Ray Aiming strategies in Optiland. 
By default, Optiland uses **Paraxial** ray aiming, which estimates ray launch coordinates based on the paraxial entrance pupil. 
For systems with significant pupil aberration (wide angle, fisheye, etc.), rays may fail to pass through the stop if aimed paraxially.

We introduce **Iterative** and **Robust** (Pupil Expansion) aiming to solve this.

In [ ]:
import optiland.backend as be
from optiland.optic import Optic
from optiland.materials import IdealMaterial

be.set_backend("numpy")

## 1. Create a Test System
We will create a simple system with an aperture stop placed after a strong lens, creating pupil aberration.

In [ ]:
optic = Optic()

# Add surfaces
optic.add_surface(index=0, thickness=100) # Object
optic.add_surface(index=1, radius=20, thickness=10, material=IdealMaterial(n=1.5))
optic.add_surface(index=2, is_stop=True, aperture=10.0, thickness=10)
optic.add_surface(index=3) # Image

# Define Fields
optic.set_field_type("angle")
optic.add_field(y=0)
optic.add_field(y=30) # 30 degrees - challenging!

# Define Aperture
optic.set_aperture("EPD", 10.0)
optic.add_wavelength(0.55)

## 2. Robust Ray Aiming
We can easily switch to a robust aiming strategy using `set_ray_aiming`.
This uses a pupil expansion algorithm to ensure convergence.

In [ ]:
# Enable Robust Ray Aiming
optic.set_ray_aiming(mode="robust", fractions=[0.1, 0.5, 1.0], tol=1e-8)

print(f"Current Aiming Config: {optic.ray_aiming_config}")

## 3. Verify
Now we generate rays. The `RayGenerator` will automatically use the robust strategy.

In [ ]:
from optiland.rays import RayGenerator

gen = RayGenerator(optic)
rays = gen.generate_rays(Hx=0, Hy=1, Px=0, Py=1, wavelength=0.55)

print(f"Ray Launch Coordinates: x={rays.x[0]:.4f}, y={rays.y[0]:.4f}")
print("Success! Rays generated using Robust Aiming.")